## Testing API access to S-122 data through an OGC API.

In [1]:
# something on OGC API
from owslib.ogcapi.features import Features

In [2]:
#srv='http://18.130.213.16/ogcfmsdi/'
srv='http://35.176.250.142/pygeoapi/'

## Access to OGC API via OWS lib interfaces. 
List out all the collections...

In [3]:
w = Features(srv)
feature_collections = w.feature_collections()
print(feature_collections)

['S-122WDPA_ES', 'S-122WDPA_ES_BB', 'S-122WDPA', 'S-122JNCC_ES', 'S-122JNCC_ES_BB', 'S-122JNCC', 'S122DNK_ES', 'S-122DNK', 'S-122DEU', 'S-122NLD', 'S122DEU_ES', 'S122NLD_ES', 'S-100Vector']


## timings
timings for different providers. This is geojson vs elasticsearch

In [4]:
%%time
denmark = w.collection_items('S-122NLD')

CPU times: user 135 ms, sys: 13.2 ms, total: 149 ms
Wall time: 1.73 s


In [5]:
%%time
denmark = w.collection_items('S-122NLD')

CPU times: user 131 ms, sys: 31.3 ms, total: 162 ms
Wall time: 1.68 s


## Complex attributes
they're not strings, they're JSON dictionaries, you can filter on these though....

In [6]:
print(denmark['features'][0]['properties']['featureName']['name'])

Schiermonnikoog


## How to retrieve features by relationship

Each relationship can be included as a distinct link (although we may have to group these by relationship name as there is a multiplicity of relationships between features and information types). so, it is possible to grab features, examine the inter-feature links and retrieve attribution from those links. This example retrieves the associated Authority feature and looks for the categoryOfAuthority it contains.

In [8]:
%%time
denmark2 = w.collection_items('S-100Vector')
print(denmark2['features'][0]['id'])
print(denmark2['features'][0]['links'])
print( (denmark2['features'][0]['links']).keys() )
relatedlink = denmark2['features'][0]['links']['protectedAreaAuthority']
print('->' + relatedlink)
d3 = w.collection_item('TEST',str(relatedlink))
print(d3['properties']['categoryOfAuthority'])


DK00VA308


KeyError: 'links'

## Filtering.
This retrieves features according to the IUCN category. Note that the category is not conformant with the S-122 enumeration values (yet). It would be possible to read in a JSON version of the feature catalogue and establish what these are (next step in this notebook).

In [9]:
%%time
denmark4 = w.collection_items('S-122DNK', categoryOfMarineProtectedArea='IV')
print(denmark4['numberMatched'])
print(denmark4['features'][1]['properties']['featureName'])

germany4 = w.collection_items('S-122DEU', categoryOfMarineProtectedArea='IV')
print(germany4['numberMatched'])
print(germany4['features'][1]['properties']['featureName'])

26
{'name': 'Ulvshale', 'language': 'DNK'}
66
{'name': 'Insel Vilm', 'language': 'DEU'}
CPU times: user 137 ms, sys: 38 ms, total: 175 ms
Wall time: 3.25 s


In [10]:
%%time
denmark1 = w.collection_items('S122DNK_ES',status='permanent')
qs =  w.collection_queryables('S122DNK_ES')

print(denmark1['features'][0])
#denmark2 = w.collection_items('TEST2', jurisdiction='international')
#print(denmark2['features'][0])


{'type': 'Feature', 'id': 'DK:H14:0', 'properties': {'featureType': 'MarineProtectedArea', 'dateEnacted': '2018-11-01T00:00:00', 'foid': 'DK:H14:0', 'featureName': {'name': 'Adler Grund og Rønne Banke'}, 'categoryOfMarineProtectedArea': 'Not Applicable', 'status': 'permanent'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[14.4136267768444, 54.812479750607], [14.1719041528197, 54.8124797761894], [14.459820942417, 55.0353302182999], [14.6072587973987, 54.9710569447474], [14.4136267768444, 54.812479750607]]]}}
CPU times: user 5.41 ms, sys: 0 ns, total: 5.41 ms
Wall time: 241 ms


## Some tests with the new server and CQL filters/queries....

In [12]:
import urllib
from urllib.request import urlopen
import json

coll='S122DNK'
qry='filter-lang=cql-text&filter=CONTAINS(geometry, POLYGON((108.58 54.19, 108.37 54.04, 108.48 53.94, 108.77 54.01, 108.77 54.11, 108.58 54.19)))'
hosttest2='http://35.176.250.142:5000/collections/' + coll + 'test2/items?f=json&' + qry

qry_body='CONTAINS(geometry, POLYGON((108.58 54.19, 108.37 54.04, 108.48 53.94, 108.77 54.01, 108.77 54.11, 108.58 54.19)))'
qry_pre='filter-lang=cql-text&filter='
srvhost='http://35.176.250.142:5000/collections/' + coll + '/items'
srvtest2=srvhost + qry_pre + qry_body

qr={'f':'json',
    'filter-lang':'cql-text',
    'filter':'WITHIN(geometry, POLYGON((6.0 57.0,9.0 57.0,9.0 54.0,6.0 54.0,6.0 57.0)))'
   }

qrys=srvhost + '?' + urllib.parse.urlencode(qr)

srvlakes_s="http://35.176.250.142:5000/collections/lakes/items?f=json"
srvlakes_t="http://35.176.250.142:5000/collections/lakes/items?f=json&filter-lang=cql-text&filter=CONTAINS(geometry,%20POLYGON((108.58%2054.19,%20108.37%2054.04,%20108.48%2053.94,%20108.77%2054.01,%20108.77%2054.11,%20108.58%2054.19)))"

srvtest3_t="http://35.176.250.142:5000/collections/test3/items?f=html&filter-lang=cql-text&filter=CONTAINS(geometry,%20POLYGON((8.0%2057.0,%2016.0%2057.0,%2016.0%2052.0,%208.0%2052.0,%208.0%2057.0)))"

#print(srvtest3_t)

theserve=qrys

print(theserve)
response = urlopen(theserve)

data_json = json.loads(response.read())

print(len(data_json['features']) , ' features found')




http://35.176.250.142:5000/collections/S122DNK/items?f=json&filter-lang=cql-text&filter=WITHIN%28geometry%2C+POLYGON%28%286.0+57.0%2C9.0+57.0%2C9.0+54.0%2C6.0+54.0%2C6.0+57.0%29%29%29
5  features found


## Testing Polygon intersect queries

This queries a bounding box made from a polygon. This uses the port 5000 version of the pygeoapi server, made with the CQL extensions enabled.

In [13]:
import urllib
from urllib.request import urlopen
import json


srvhost='http://35.176.250.142:5000/collections/S122DNK/items'

qr={'f':'json','filter-lang':'cql-text','filter':'WITHIN(geometry,POLYGON((6.0 57.0,9.0 57.0,9.0 54.0,6.0 54.0,6.0 57.0)))'}
qrys=srvhost + '?' + urllib.parse.urlencode(qr)

theserve=qrys
print(theserve)

response = urlopen(theserve)
data_json = json.loads(response.read())

print(len(data_json['features']) , ' features found')


http://35.176.250.142:5000/collections/S122DNK/items?f=json&filter-lang=cql-text&filter=WITHIN%28geometry%2CPOLYGON%28%286.0+57.0%2C9.0+57.0%2C9.0+54.0%2C6.0+54.0%2C6.0+57.0%29%29%29
5  features found


In [14]:
srvhost='http://35.176.250.142/pygeoapi/collections/S122DNK_ES/items'

qr={'f':'json',
    'bbox':'6.0,54.0,9.0,57.0'}
qrys=srvhost + '?' + urllib.parse.urlencode(qr)

theserve=qrys
print(theserve)

response = urlopen(theserve)
data_json = json.loads(response.read())

print(len(data_json['features']) , ' features found')

http://35.176.250.142/pygeoapi/collections/S122DNK_ES/items?f=json&bbox=6.0%2C54.0%2C9.0%2C57.0
5  features found
